### Keepass Specific
- Only a password is needed
- All references to a username in code/functions removed

In [13]:
def variables():
    KEEPASS_PATH = ReadINI("KEEPASS_PATH")
    KEEPASS_PATH_ALT = ReadINI("KEEPASS_PATH_ALT")
    KEEPASS_PATH_SYS = ReadINI("KEEPASS_PATH_SYS")
    KPFileName = ReadINI("KEEPASS_APP")

    KP1_PATH = ReadINI("KP1_PATH")
    KP1_FILE = ReadINI("KP1_FILE")
    KP1_ENCRYPT_HASH = ReadINI("KP1_ENCRYPT_HASH")

    KP2_PATH = ReadINI("KP2_PATH")
    KP2_FILE = ReadINI("KP2_FILE")
    KP2_ENCRYPT_HASH = ReadINI("KP2_ENCRYPT_HASH")

    KP3_PATH = ReadINI("KP3_PATH")
    KP3_FILE = ReadINI("KP3_FILE")
    KP3_ENCRYPT_HASH = ReadINI("KP3_ENCRYPT_HASH")
    KP3_KEY_PATH = ReadINI("KP3_KEY_PATH")
    KP3_KEY = ReadINI("KP3_KEY")

In [14]:
# Allows CMD line statements to be run
import os

# Allows passwords to be input without revealing them
import getpass

# Tool to encrypt text (passwords) // not designed for encrypt, just result matching
from passlib.hash import pbkdf2_sha256

# Tool to encrypt/decrypt text (Keepass passwords)
# pip install pycryptodome
from Crypto.Cipher import AES

# Tool to read/write config text files
try:
    from configparser import ConfigParser
except ImportError:
    from ConfigParser import ConfigParser  # ver. < 3.0

In [34]:
def createini(password):
    ini_file_name = 'kp_config.ini' 
    
    # instantiate
    config = ConfigParser()
    
    # parse existing file
    with open(ini_file_name, 'w') as configfile:
        config.write(configfile)

    # parse existing file
    config.read(ini_file_name)

    # add a new section and some values
    config.add_section('KEEPASS_CREDENTIALS')
    config.set('KEEPASS_CREDENTIALS', 'KP1_HASH', password)
    #config.set('ENTRY 1', 'user1', username)

    with open(ini_file_name, 'w') as configfile:
        config.write(configfile)
        
def verify():
    pw_verify = getpass.getpass("Confirm your password: ")

    # instantiate
    config = ConfigParser()
    
    # parse existing file
    config.read('kp_config.ini')
    
    # read values from a section
    read_hash = config.get('KEEPASS_CREDENTIALS', 'KP1_HASH')
    #read_user = config.get('ENTRY 1', 'user1')
    #int_val = config.getint('section_a', 'int_val')
    #float_val = config.getfloat('section_a', 'pi_val')
    i = 0
    while ((pbkdf2_sha256.verify(pw_verify, read_hash) == False) & (i < 5)):
        print("\nUsername or password incorrect, please try again.")
        pw_verify = getpass.getpass("Confirm your password: ")
        i += 1
        #while (pbkdf2_sha256.verify(pw_verify, read_hash) == False):
    if (pbkdf2_sha256.verify(pw_verify, read_hash) == True):       
        print("\nPassword successful.")
    else:
        print("Too many tries. Please start over.")
        
def login(password):   
    # instantiate
    config = ConfigParser()
    
    # parse existing file
    config.read('kp_config.ini')
    
    # read values from a section
    read_hash = config.get('KEEPASS_CREDENTIALS', 'KP1_HASH')
    #read_user = config.get('ENTRY 1', 'user1')
    
    i = 0
    while ((pbkdf2_sha256.verify(password, read_hash) == False) & (i < 5)):
        print("\nUsername or password incorrect, please try again.")
        password = getpass.getpass("Confirm your password: ")
        i += 1
        #while (pbkdf2_sha256.verify(pw_verify, read_hash) == False):
    
    if (pbkdf2_sha256.verify(password, read_hash) == True):       
        print("\n\nSuccess.")
    else:
        print("Too many tries. Please start over.")
        
def encrypt_password(password):
    return pbkdf2_sha256.hash(password, rounds=500000, salt_size=16)

def test_password():
    next_step = ""
    while ((next_step != "yes") & (next_step != "no")):
        next_step = input("Do you want to test login as well? (yes/no)")
        if (next_step == "yes"):
            login_pw = getpass.getpass("Password: ")
            login(login_pw)
        elif (next_step == "no"):
            print("Test is complete then.")
        else:
            print("Input must be 'yes' or 'no'.")
        
def main():
    password_hash = encrypt_password(getpass.getpass("What is your password?"))
    createini(password_hash)
    verify()
    test_password()
    

In [35]:
def keepass_login(keepass_account):
    ini_file = "kp_config_logins.ini"
    account_path = keepass_account + "_PATH"
    account_file = keepass_account + "_FILE"
    account_hash = keepass_account + "_ENCRYPT_HASH"
    
    # instantiate
    config = ConfigParser()

    # parse existing file
    config.read('kp_config_logins.ini')

    # read values
    path = config.get('KEEPASS_CREDENTIALS', account_path)
    print(path)
    file = config.get('KEEPASS_CREDENTIALS', account_file)
    print(file)
    hash = config.get('KEEPASS_CREDENTIALS', account_hash)
    print(hash)
    keepass_path = config.get('KEEPASS_EXE', 'KEEPASS_PATH')
    keepass_file = config.get('KEEPASS_EXE', 'KEEPASS_APP')
    
    print((chr(34) + keepass_path + keepass_file + chr(34) + chr(34) + path + file + chr(34)))
    
    # "hash" needs to be replaced by the decrypt function... decypt_hash(hash)
    # instruction comes from: https://keepass.info/help/base/cmdline.html
    os.system(keepass_path + keepass_file + " " + chr(34) + path + file + chr(34) + " -pw:" + hash)

In [29]:
keepass_login("KP1")

/
test.kdbx

"C:\KeePass-Modified\Keepass.exe""/test.kdbx"


In [36]:
main()

What is your password? ············
Confirm your password:  ············



Password successful.


Do you want to test login as well? (yes/no) yes
Password:  ············




Success.


In [ ]:
login_pw = getpass.getpass("Password: ")
login(login_pw)

In [ ]:
os.system('')